In [ ]:
import os
import pandas as pd
import numpy as np

from src.util import *
from src.sarima import SARIMAModel

In [ ]:
ROOT = os.getcwd()
MODELS = os.path.join(ROOT, 'models')
METRICS = os.path.join(ROOT, 'metrics')
DATA = os.path.join(ROOT, 'dataset')
FIGURES = os.path.join(ROOT, 'figures')

## Construir Dataset

In [ ]:
# Read the CSV file from the dataset folder
file_path = os.path.join(DATA, 'TrafficTwoMonth.csv')
df = pd.read_csv(file_path)

# Create datetime column
df = create_datetime_column(df)
df.drop(['Time', 'Date', 'Day of the week'], axis=1, inplace=True)

## Análisis

In [ ]:
analyser = TrafficAnalyzer(df)

message = analyser.check_stationarity(df['Total'])
print(message)

analyser.plot_time_series(
    figsize=(10, 6), 
    columns=['Total', 'CarCount', 'BikeCount'], 
    xticks_step=96, 
    start_pct=20, 
    end_pct=30,
    save_path=os.path.join(FIGURES, 'time_series_traffic.png'))

analyser.plot_autocorrelation(series=df['Total'], 
                              lags=20, 
                              save_path=os.path.join(FIGURES, 'autocorrelation_traffic.png'))

## SARIMA

### Introducción teórica

El modelo SARIMA (Seasonal AutoRegressive Integrated Moving Average) es una extensión del modelo ARIMA que incorpora la estacionalidad. Se expresa matemáticamente como SARIMA(p,d,q)(P,D,Q,m) donde:

- p: orden de la parte autorregresiva (AR)
- d: grado de diferenciación necesario para alcanzar la estacionariedad
- q: orden de la parte de media móvil (MA)
- P: orden de la parte autorregresiva estacional (SAR)
- D: grado de diferenciación estacional
- Q: orden de la parte de media móvil estacional (SMA)
- m: período estacional

La ecuación general del modelo SARIMA es:

$\Phi_P(L^m)\phi_p(L)(1-L^m)^D(1-L)^dy_t = \Theta_Q(L^m)\theta_q(L)\epsilon_t$

Donde:
- $L$ es el operador de retardo
- $\phi_p(L)$ es el operador AR no estacional
- $\Phi_P(L^m)$ es el operador AR estacional
- $\theta_q(L)$ es el operador MA no estacional
- $\Theta_Q(L^m)$ es el operador MA estacional
- $\epsilon_t$ es el término de error

### Secuencia de Modelado

1. Primero creamos un modelo SARIMA usando los datos de tráfico total
2. Optimizamos los parámetros del modelo probando diferentes combinaciones de p, d, q y P, D, Q (con m=96)
3. Ajustamos el modelo con los mejores parámetros encontrados
4. Realizamos predicciones usando rolling forecast con una ventana de entrenamiento del 95% de los datos
5. Visualizamos los resultados comparando las predicciones con los datos reales

In [ ]:
p_range = range(1, 2)
q_range = range(1, 2)
P_range = range(1, 3)
Q_range = range(1, 3)
m = 12
d = 0
D = 1

In [ ]:
# Optimización
sarima_model = SARIMAModel(df['Total'])
sarima_model.optimize(p_range, d, q_range, P_range, D, Q_range, m)
sarima_model.fit(order=sarima_model.best_order, seasonal_order=sarima_model.best_seasonal_order)

In [ ]:
# Predicción
train_size = int(len(df) * 0.98)
window = 2
predictions_sarima = sarima_model.rolling_forecast(train_size, window)

In [ ]:
# Visualización
start_i = int(len(df) * 0.90)
end_i = int(len(df) * 0.98)
sarima_model.plot_predictions(
    train_data=df['Total'].values[start_i:end_i],
    test_data=df['Total'].values[end_i:],
    predictions=predictions_sarima,
    title='Predicciones de tráfico total - SARIMA',
    save_path=os.path.join(FIGURES, 'sarima_predictions.png')
)

In [ ]:
# Métricas
test_data = np.array(df['Total'].iloc[int(len(df) * 0.98):])
predictions_sarima = np.array(predictions_sarima)
sarima_metrics = sarima_model.calculate_metrics(test_data, predictions_sarima)

for key, value in sarima_metrics.items():
    print(f"{key}: {value:.4f}")